This exercise will require you to pull some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [11]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '' # your Quandl API key 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [12]:
# First, import the relevant modules
import requests

In [13]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
payload = {'start_date': '2019-02-26', 'end_date': '2019-02-26', 'api_key': API_KEY}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json', params=payload)

In [14]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{u'dataset': {u'collapse': None,
  u'column_index': None,
  u'column_names': [u'Date',
   u'Open',
   u'High',
   u'Low',
   u'Close',
   u'Change',
   u'Traded Volume',
   u'Turnover',
   u'Last Price of the Day',
   u'Daily Traded Units',
   u'Daily Turnover'],
  u'data': [[u'2019-02-26',
    None,
    76.8,
    74.85,
    76.25,
    None,
    118344.0,
    9005952.0,
    None,
    None,
    None]],
  u'database_code': u'FSE',
  u'database_id': 6129,
  u'dataset_code': u'AFX_X',
  u'description': u'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  u'end_date': u'2019-02-26',
  u'frequency': u'daily',
  u'id': 10095370,
  u'limit': None,
  u'name': u'Carl Zeiss Meditec (AFX_X)',
  u'newest_available_date': u'2019-02-27',
  u'oldest_available_date': u'2000-06-07',
  u'order': None,
  u'premium': False,
  u'refreshed_at': u'2019-02-27T22:30:34.021Z',
  u'start_date': u'2019-02-26',
  u'transform': None

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

****
### 1. Collect data for all of 2017

In [15]:
payload = {'start_date': '2017-01-01', 'end_date': '2017-12-31', 'api_key': API_KEY}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json', params=payload)

### 2. Convert JSON to dictionary

In [16]:
data = r.json()
print(type(data))

<type 'dict'>


### 3. Calculate the highest and lowest opening prices
The opening price is the 2nd column of the data.

In [22]:
# Construct list of opening prices and filter out null values
open_price = [entry[1] for entry in data['dataset']['data'] if entry[1] is not None]

print('max opening price: ' + str(max(open_price)) + '\n' + 'min opening price: ' + str(min(open_price)))

max opening price: 53.11
min opening price: 34.0


### 4. Calculate the largest change in any one day
The high and low prices are the 3rd and 4th columns of the data, respectively.

In [23]:
# Construct list of change prices for each day, filtering out days with null values
change_price = [entry[2] - entry[3] for entry in data['dataset']['data'] if entry[2] is not None and entry[3] is not None]

print('max change in one day: ' + str(max(change_price)))

max change in one day: 2.81


### 5. Calculate the largest change between any two days
The closing price is the 5th column of the data.

In [36]:
# Construct list of closing prices and filter out null values
close_price = [entry[4] for entry in data['dataset']['data'] if entry[4] is not None]

# Calculate magnitude of closing price changes between days
close_price_diff = [abs(close_price[i] - close_price[i - 1]) for i, _ in enumerate(close_price) if i > 0]

print('max change between two days: ' + str(max(close_price_diff)))

max change between two days: 2.56


### 6. Calculate average daily trading volume
The trade volume is the 7th column in the data.

In [37]:
# Construct list of trading volumes and filter out null values
trade_volume = [entry[6] for entry in data['dataset']['data'] if entry[6] is not None]

avg_volume = sum(trade_volume) / len(trade_volume)
print('average daily trading volume: ' + str(avg_volume))

average daily trading volume: 89124.3372549


### 7. Calculate median daily trading volume
Could use numpy or statistics library to calculate median, but will just define own function here.

In [46]:
def median(lst):
    """Calculates the median of a list of ints or floats."""
    lst = sorted(lst)
    n = len(lst)
    if n % 2 == 0:
        return (lst[n / 2 - 1] + lst[n / 2]) / 2.0
    else:
        return lst[n / 2]

print('median daily trading volume: ' + str(median(trade_volume)))

median daily trading volume: 76286.0
